# What are Embedding Layers in Keras

[Embedding Layers](https://keras.io/layers/embeddings/) are a handy feature of Keras that allows the program to automatically insert additional information into the data flow of your neural network. In the previous section, you saw that Word2Vec could expand words to a 300 dimension vector. An embedding layer would automatically allow you to insert these 300-dimension vectors in the place of word indexes.  

Programmers often use embedding layers with Natural Language Processing (NLP); however, you can use these layers when you wish to insert a lengthier vector in an index value place. In some ways, you can think of an embedding layer as dimension expansion. However, the hope is that these additional dimensions provide more information to the model and provide a better score.

## Simple Embedding Layer Example

* **input_dim** = How large is the vocabulary?  How many categories are you encoding? This parameter is the number of items in your "lookup table."
* **output_dim** = How many numbers in the vector you wish to return. 
* **input_length** = How many items are in the input feature vector that you need to transform?

Now we create a neural network with a vocabulary size of 10, which will reduce those values between 0-9 to 4 number vectors. This neural network does nothing more than passing the embedding on to the output. But it does let us see what the embedding is doing. Each feature vector coming in will have two such features.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

model = Sequential()
embedding_layer = Embedding(input_dim=10, output_dim=4, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')


Let's take a look at the structure of this neural network to see what is happening inside it.

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 4)              40        
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


For this neural network, which is just an embedding layer, the input is a vector of size 2. These two inputs are integer numbers from 0 to 9 (corresponding to the requested input_dim quantity of 10 values). Looking at the summary above, we see that the embedding layer has 40 parameters. This value comes from the embedded lookup table that contains four amounts (output_dim) for each of the 10 (input_dim) possible integer values for the two inputs. The output is 2 (input_length) length 4 (output_dim) vectors, resulting in a total output size of 8, which corresponds to the Output Shape given in the summary above.

Now, let us query the neural network with two rows. The input is two integer values, as was specified when we created the neural network.

In [3]:
input_data = np.array([
    [1, 2]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)


(1, 2)
[[[ 0.0296591  -0.03944106 -0.00196435  0.0168138 ]
  [ 0.04712534 -0.0231918  -0.04047847 -0.04747871]]]


Here we see two length-4 vectors that Keras looked up for each input integer. Recall that Python arrays are zero-based. Keras replaced the value of 1 with the second row of the 10 x 4 lookup matrix. Similarly, Keras returned the value of 2 by the third row of the lookup matrix. The following code displays the lookup matrix in its entirety. The embedding layer performs no mathematical operations other than inserting the correct row from the lookup table.

In [4]:
embedding_layer.get_weights()

[array([[-1.59338489e-02, -6.17409870e-03, -3.28724608e-02,
         -1.48969665e-02],
        [ 2.96590962e-02, -3.94410603e-02, -1.96435302e-03,
          1.68138035e-02],
        [ 4.71253432e-02, -2.31917985e-02, -4.04784679e-02,
         -4.74787131e-02],
        [-1.38375759e-02,  4.91913073e-02,  1.64018162e-02,
          2.43717097e-02],
        [ 1.40725859e-02,  2.42561139e-02,  4.45892662e-03,
          4.25609611e-02],
        [ 2.11127512e-02, -4.18646932e-02, -4.76553701e-02,
         -2.57306583e-02],
        [-2.26217862e-02, -4.80080768e-03,  4.77720387e-02,
          4.04601954e-02],
        [-3.18327770e-02,  3.72967608e-02,  4.22089733e-02,
          3.46850231e-03],
        [-2.50756145e-02,  2.05684789e-02,  2.10559852e-02,
          2.22275518e-02],
        [ 2.21610069e-03, -9.68669727e-03,  3.06015350e-02,
         -5.12972474e-05]], dtype=float32)]

The values above are random parameters that Keras generated as starting points.  Generally, we will transfer an embedding or train these random values into something useful.  The following section demonstrates how to embed a hand-coded embedding. 

## Transferring An Embedding

Now, we see how to hard-code an embedding lookup that performs a simple one-hot encoding.  One-hot encoding would transform the input integer values of 0, 1, and 2 to the vectors $[1,0,0]$, $[0,1,0]$, and $[0,0,1]$ respectively. The following code replaced the random lookup values in the embedding layer with this one-hot coding-inspired lookup table.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

embedding_lookup = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

model = Sequential()
embedding_layer = Embedding(input_dim=3, output_dim=3, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')

embedding_layer.set_weights([embedding_lookup])


We have the following parameters for the Embedding layer:
    
* input_dim=3 - There are three different integer categorical values allowed.
* output_dim=3 - Three columns represent a categorical value with three possible values per one-hot encoding.
* input_length=2 - The input vector has two of these categorical values.

We query the neural network with two categorical values to see the lookup performed.

In [6]:
input_data = np.array([
    [0, 1]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)


(1, 2)
[[[1. 0. 0.]
  [0. 1. 0.]]]


The given output shows that we provided the program with two rows from the one-hot encoding table. This encoding is a correct one-hot encoding for the values 0 and 1, where there are up to 3 unique values possible. 

The following section demonstrates how to train this embedding lookup table.

## Training an Embedding

First, we make use of the following imports.

In [7]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

We create a neural network that classifies restaurant reviews according to positive or negative.  This neural network can accept strings as input, such as given here.  This code also includes positive or negative labels for each review.

In [8]:
# Define 10 resturant reviews.
reviews = [
    'Never coming back!',
    'Horrible service',
    'Rude waitress',
    'Cold food.',
    'Horrible food!',
    'Awesome',
    'Awesome service!',
    'Rocks!',
    'poor work',
    'Couldn\'t have done better']

# Define labels (1=negative, 0=positive)
labels = array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


Notice that the second to the last label is incorrect.  Errors such as this are not too out of the ordinary, as most training data could have some noise.

We define a vocabulary size of 50 words.  Though we do not have 50 words, it is okay to use a value larger than needed.  If there are more than 50 words, the least frequently used words in the training set are automatically dropped by the embedding layer during training.  For input, we one-hot encode the strings.  We use the TensorFlow one-hot encoding method here rather than Scikit-Learn. Scikit-learn would expand these strings to the 0's and 1's as we would typically see for dummy variables.  TensorFlow translates all words to index values and replaces each word with that index.

In [9]:
VOCAB_SIZE = 50
encoded_reviews = [one_hot(d, VOCAB_SIZE) for d in reviews]
print(f"Encoded reviews: {encoded_reviews}")


Encoded reviews: [[9, 42, 7], [14, 9], [23, 21], [45, 21], [14, 21], [35], [35, 9], [9], [12, 10], [27, 15, 11, 19]]


The program one-hot encodes these reviews to word indexes; however, their lengths are different.  We pad these reviews to 4 words and truncate any words beyond the fourth word.

In [10]:
MAX_LENGTH = 4

padded_reviews = pad_sequences(encoded_reviews, maxlen=MAX_LENGTH,
                               padding='post')
print(padded_reviews)


[[ 9 42  7  0]
 [14  9  0  0]
 [23 21  0  0]
 [45 21  0  0]
 [14 21  0  0]
 [35  0  0  0]
 [35  9  0  0]
 [ 9  0  0  0]
 [12 10  0  0]
 [27 15 11 19]]


As specified by the **padding=post** setting, each review is padded by appending zeros at the end, as specified by the **padding=post** setting.

Next, we create a neural network to learn to classify these reviews. 

In [11]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 8, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['acc'])

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


This network accepts four integer inputs that specify the indexes of a padded movie review. The first embedding layer converts these four indexes into four length vectors 8. These vectors come from the lookup table that contains 50 (VOCAB_SIZE) rows of vectors of length 8. This encoding is evident by the 400 (8 times 50) parameters in the embedding layer. The output size from the embedding layer is 32 (4 words expressed as 8-number embedded vectors). A single output neuron is connected to the embedding layer by 33 weights (32 from the embedding layer and a single bias neuron). Because this is a single-class classification network, we use the sigmoid activation function and binary_crossentropy.

The program now trains the neural network. The embedding lookup and dense 33 weights are updated to produce a better score.

In [12]:
# fit the model
model.fit(padded_reviews, labels, epochs=100, verbose=0)


We can see the learned embeddings.  Think of each word's vector as a location in the 8 dimension space where words associated with positive reviews are close to other words.  Similarly, training places negative reviews close to each other.  In addition to the training setting these embeddings, the 33 weights between the embedding layer and output neuron similarly learn to transform these embeddings into an actual prediction.  You can see these embeddings here.

In [13]:
print(embedding_layer.get_weights()[0].shape)
print(embedding_layer.get_weights())


(50, 8)
[array([[ 1.35023564e-01, -1.02895461e-01, -1.19495116e-01,
        -1.29351303e-01, -7.90727660e-02,  9.17939395e-02,
         9.20079201e-02,  1.14221409e-01],
       [-4.78599221e-03,  3.80156897e-02, -2.85736471e-03,
         1.62495486e-02,  2.48310827e-02,  8.74362886e-04,
        -1.90549847e-02, -4.14300710e-04],
       [-3.64602096e-02,  5.03417104e-03, -4.41656262e-03,
         4.72053029e-02,  3.68510000e-02, -1.15816705e-02,
        -3.92480940e-03, -1.33359544e-02],
       [-3.43585610e-02,  4.57068197e-02, -4.18290012e-02,
        -2.10601576e-02, -1.94263458e-02, -4.92715500e-02,
         2.45969929e-02, -1.72461756e-02],
       [-4.08391580e-02,  4.23473231e-02,  1.38023011e-02,
        -1.48387179e-02, -1.86042674e-02, -2.52524149e-02,
         2.61281393e-02, -2.42151506e-02],
       [-3.46100554e-02, -3.37571502e-02, -3.83336768e-02,
         3.23278196e-02,  2.34506242e-02,  3.54191698e-02,
         2.99644209e-02, -4.52305563e-02],
       [-8.44417885e-03, 

We can now evaluate this neural network's accuracy, including the embeddings and the learned dense layer.  

In [14]:
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print(f'Accuracy: {accuracy}')


Accuracy: 1.0


The accuracy is a perfect 1.0, indicating there is likely overfitting. It would be good to use early stopping to not overfit for a more complex data set.

In [15]:
print(f'Log-loss: {loss}')


Log-loss: 0.45874184370040894


However, the loss is not perfect. Even though the predicted probabilities indicated a correct prediction in every case, the program did not achieve absolute confidence in each correct answer. The lack of confidence was likely due to the small amount of noise (previously discussed) in the data set. Some words that appeared in both positive and negative reviews contributed to this lack of absolute certainty. 